In [7]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

#support of dataset retrival with hugging face
from datasets import load_dataset

import cassio

In [8]:
from PyPDF2 import PdfReader

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
    

Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15


True

In [10]:
#provide the path of pdf file
pdfreader = PdfReader("../YogiMeterFeeler222F(32.1).pdf")
pdfreader.pages

In [11]:
from typing_extensions import Concatenate

raw_text = ""
for i, page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text += content
        
raw_text

'73.2%26.8%\nFEELING/THINKING SOLVING/LEARNING\nYogiMeter analyses your Choice Making Mind and projects your decision making according to\nyour current Mindset. Learn to understand and change your mindset to become your best self.\nCAS (Cognitive Attention Score)\nCurrently your feeling mind is running your life. You are either spoilt for choices or just slow and confused. Your\nmind makes you spend too much time in the future. Your mind is playing tricks on you. It does not let you focus\nand does not let you progress or �nd peace. Your focus is also compromised.1 days\n2 days\n32 days\n18 days\n7 days\nYogiMeter has detected your general and leadership mental wellness scores.\nUpcoming 60 daysSometimes your mind is improving your life and sometimes it is spoiling it.\nYou are taking your emotions too seriously and they are pulling you down.\nEmpathy Tendency⇢\nConsistency & Persistence Tendency⇢\nLiving in Present Moment Tendency⇢\nTimeliness Tendency⇢\nCode of Conduct Tendency⇢\nInn

In [12]:
#init db
cassio.init(token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"), database_id=os.getenv("ASTRA_DB_ID"))

In [13]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

/tmp/ipykernel_35618/2378891225.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
/tmp/ipykernel_35618/2378891225.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))


### Langchain vector store db astra db

In [14]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="pdf_query_yogi_meter",
    session=None,
    keyspace=None
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************0v0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [44]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap = 200,
    length_function = len
)

texts = text_splitter.split_text(raw_text)
texts[:10]

['73.2%26.8%\nFEELING/THINKING SOLVING/LEARNING\nYogiMeter analyses your Choice Making Mind and projects your decision making according to\nyour current Mindset. Learn to understand and change your mindset to become your best self.\nCAS (Cognitive Attention Score)\nCurrently your feeling mind is running your life. You are either spoilt for choices or just slow and confused. Your\nmind makes you spend too much time in the future. Your mind is playing tricks on you. It does not let you focus\nand does not let you progress or �nd peace. Your focus is also compromised.1 days\n2 days\n32 days\n18 days\n7 days\nYogiMeter has detected your general and leadership mental wellness scores.\nUpcoming 60 daysSometimes your mind is improving your life and sometimes it is spoiling it.',
 '2 days\n32 days\n18 days\n7 days\nYogiMeter has detected your general and leadership mental wellness scores.\nUpcoming 60 daysSometimes your mind is improving your life and sometimes it is spoiling it.\nYou are taki

In [46]:
#load the dataset into vector db
astra_vector_store.add_texts(texts[:50])

astra_vector_index= VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [48]:
first_question = True
while True:
    if first_question:
        query_text = input("Enter your question: ").strip()
    else:
        query_text = input("what is your next question (type quit or exit): ").strip()
    
    if query_text.lower() == 'quit':
        break
    if query_text.lower() == '':
        continue
    
    first_question = False
    
    print("\n Question: \%s\"" %query_text)
    answers = astra_vector_index.query(query_text, llm=llm).strip()
    print("Answer: \"%s\"\n" % answers)
    
    print("First similar questions:")
    
    for doc , score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("[%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


 Question: \what is yogi"
Answer: "Yogi is a term used to describe someone who practices yoga and is dedicated to achieving physical, mental, and spiritual balance and harmony. In this context, YogiMeter is a tool that helps individuals assess their mental wellness and make positive changes to improve their overall well-being."

First similar questions:
[0.8784] "2 days
32 days
18 days
7 days
YogiMeter has detected your general and leadership men ..."
[0.8758] "73.2%26.8%
FEELING/THINKING SOLVING/LEARNING
YogiMeter analyses your Choice Making M ..."
[0.8717] "Decisiveness Tendency⇢
Pursuit of Excellence Tendency⇢
Enthusiasm Tendency⇢
Confusio ..."
[0.8683] "This dosha indicates that the person is likely to use force to accomplish
their goal ..."

 Question: \exit"
Answer: "I don't know, I cannot find any mention of "exit" in the given context. Could you please provide more information?"

First similar questions:
[0.8812] "that it will justify the expenditure.2-30 3 6
Your habits indic